In [1]:
# TO DO: Create validation report

In [2]:
import cx_Oracle
import pandas as pd
from datetime import timedelta
from datetime import datetime   
import numpy as np
import os
import re

In [3]:
# Initializing database connection
con = cx_Oracle.connect('STC_SETRAG/art@localhost/orcl.eng.engesis.com')
cur = con.cursor()

    # First Query
query=cur.execute('SELECT * FROM HISTORICO_MENSAGENS')
df = pd.DataFrame(query)
    # Second Query

cur.close()
con.close()

In [4]:
df.columns=['HM_ID_HM','HM_LOCO_MSG','HM_PRF_TRM','HM_OBC_MSG','HM_NUM_MSG','HM_TP_MSG','HM_DAT_OBC','HM_DAT_STC', 'HM_TXT_MSG','HM_TXT_TW','HM_LAT_HM','HM_LON_HM','HM_ENV_RCB','HM_MSG_VOZ','HM_NUM_FUN','HM_ID_AUX','HM_SIT_ENV','MT_ID_MT']
df = df.drop(['HM_TP_MSG','HM_TXT_TW','HM_ID_AUX','HM_OBC_MSG','HM_LAT_HM','HM_LON_HM'], axis=1)

df_sent = df.loc[df['HM_ENV_RCB']=='E']

df_received = df.loc[df['HM_ENV_RCB']=='R']

    # Removing the original dataframe
df = pd.DataFrame(None)

In [5]:
month = '2'
year = '2021'
a = timedelta(seconds=300)

In [6]:
df_sent = df_sent.loc[df_sent['HM_DAT_STC']>= year + '-' + month +'-12']
if(month == '12'):     # December
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']< str(int(year)+1) + '-1-01']
else:                  # Other months
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']< year + '-' + str(int(month)) +'-13']
    # 2) Filter all messages that contain OBC time
df_sent = df_sent.loc[df_sent['HM_DAT_OBC'].notnull()]
    # 3) Filter messages to ensure that the STC time is smaller (or igual) to the OBC time (avoid negative values)
df_sent = df_sent.loc[df_sent['HM_DAT_STC']<=df_sent['HM_DAT_OBC']]
    # 4) Filter messages that contain locomotive ID
df_sent = df_sent[df_sent['HM_LOCO_MSG'].notnull()]

    # Received DataFrame
    # 1) Filter all messages that did not expired
df_received = df_received.loc[df_received['HM_SIT_ENV']=='E']
    # 2) Filter messages from within the date set by user
df_received = df_received.loc[df_received['HM_DAT_STC']>= year + '-' + month +'-01']
if(month == '12'):     # December
    df_received = df_received.loc[df_received['HM_DAT_STC']< str(int(year)+1) + '-1-01']
else:                  # Other months
    df_received = df_received.loc[df_received['HM_DAT_STC']< year + '-' + str(int(month)+1) +'-01']
    # 3) Filter all messages that contain OBC time
df_received = df_received.loc[df_received['HM_DAT_OBC'].notnull()]
    # 4) Filter all messages that have a date later than needed (avoiding future date messages)
if(month == '12'):
    df_received = df_received.loc[df_received['HM_DAT_OBC']< str(int(year)+1) +'-1-02']
else:
    df_received = df_received.loc[df_received['HM_DAT_OBC']< year + '-' + str(int(month)+1) +'-02']
    # 5) Filter messages to ensure that the OBC time is smaller (or igual) to the STC time (avoid negative values)
df_received = df_received.loc[df_received['HM_DAT_OBC']<=df_received['HM_DAT_STC']]
    # 6) Filter messages that contain locomotive ID
df_received = df_received[df_received['HM_LOCO_MSG'].notnull()]

In [7]:
totalSentMessages = len(df_sent['HM_ID_HM'])
    # Received
totalReceivedMessages = len(df_received['HM_ID_HM'])
    # Total
totalMessages = totalSentMessages + totalReceivedMessages

In [8]:
    # Sent
df_sent['DELIVERY_TIME'] = df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']
    # Converting to string in order to execute a mathematic operation
df_sent['DELIVERY_TIME'] = df_sent['DELIVERY_TIME']/np.timedelta64(1, 's')
totalSent = sum(df_sent.iloc[:,12])
averageSentDelivery = round(totalSent/totalSentMessages,0)
    # Converting from float to mm:ss
averageSentDelivery = timedelta(seconds=averageSentDelivery)
averageSentDelivery=str(averageSentDelivery)

    # Received
df_received['DELIVERY_TIME'] = df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']
    # Converting to string in order to execute a mathematic operation
df_received['DELIVERY_TIME'] = df_received['DELIVERY_TIME']/np.timedelta64(1, 's')
totalReceived = sum(df_received.iloc[:,12])
averageReceivedDelivery = round(totalReceived/totalReceivedMessages,0)


In [9]:
    # Sent
    # 3) Filter messages that took longer than 5 minutes
df_sent = df_sent.loc[(df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']) >= a]
    # 4) Order the dataframe by HM_ID_HM
df_sent = df_sent.sort_values(by=['HM_ID_HM'])

    #Received
    # 5) Filter messages that took longer than 5 minutes
df_received = df_received.loc[(df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']) >= a]
    # 6) Order the dataframe by HM_ID_HM
df_received = df_received.sort_values(by=['HM_ID_HM'])

In [10]:
prefix = hex(int(df_sent.iloc[33,1]))
df_sent.iloc[33,1]

'4304'

In [11]:
import logging
# For logging
logging.basicConfig(level=logging.INFO, filename="data.log", format="%(asctime)-15s %(levelname)-8s %(message)s")

In [12]:
# Adding new columns to count channels
df_sent['GPRS'] = 0
df_sent['SAT'] = 0
df_sent['QUEUE_DELAY'] = 0

df_received['GSM'] = 0
df_received['SAT'] = 0

In [13]:
#converting it from float to INT
df_sent['MT_ID_MT']=df_sent['MT_ID_MT'].astype('int')

In [14]:
# Importing files
myfile = open("D:\\kenneth\Documents\\Clients\\SETRAG\\Tickets\\SDSTG_561_Server\\SDSTG_561\\MM\\STCMessageManager.log", errors="ignore")
mm_log = myfile.read().splitlines()
myfile.close()

myfile = open("D:\\kenneth\\Documents\\Clients\\SETRAG\\Tickets\\SDSTG_561_Server\\SDSTG_561\\app_ABR.log")
abr_log = myfile.read().splitlines()
myfile.close()

In [15]:
####
####
# HERE HAS TO BE REPEATED FOR ALL ROWS

In [16]:
def func1(i, df_sent):    
    # Converting string to MM date format
    new_format=str(df_sent.iloc[i,5])
    temp=""

    for i in new_format: 
        if i.isalnum(): # Strips all special characters
            temp += i

    new_format=temp[4:6]+'/'+temp[6:8]+'/'+temp[0:4]+' '+temp[8:10]+':'+temp[10:12]+':' # Seconds is not included because there may be a difference between DB and MM

    # To cover in case of difference in minutes (08:30 -> 08:29)
    if temp[10] != '0':
        new_format_I = temp[4:6]+'/'+temp[6:8]+'/'+temp[0:4]+' '+temp[8:10]+':'+str(int(temp[10:12])-1)+':'
    else:
        new_format_I = temp[4:6]+'/'+temp[6:8]+'/'+temp[0:4]+' '+temp[8:10]+':0'+str(int(temp[10:12])-1)+':' # To cover in case of difference in minutes (08:30 -> 08:29)

    # To cover in case of difference in hour (08:00 -> 07:59)
    if temp[8] != '0':
        new_format_II = temp[4:6]+'/'+temp[6:8]+'/'+temp[0:4]+' '+str(int(temp[8:10])-1)+':'+'59'+':' 
    elif temp[8] == '0' and temp[9] != '0':
        new_format_II = temp[4:6]+'/'+temp[6:8]+'/'+temp[0:4]+' 0'+str(int(temp[8:10])-1)+':'+'59'+':' # Without this it will change an hour '08' into '7', which doesn't work
    else:
        new_format_II = temp[4:6]+'/'+str(int(temp[6:8])-1)+'/'+temp[0:4]+' 23:59:' # In case the hour is 00:00:00


    return (new_format, new_format_I, new_format_II)

In [17]:
def func2(i, df_sent, new_format, mm_log, new_format_I, new_format_II):
    # Regex used to match relevant loglines 
    line_regex = re.compile(new_format + r'.*?' + " Mensagem Enviada - Locomotiva: "+df_sent.iloc[i,1]+" Tipo de comunicação: 1 Número Mensagem: "+str(df_sent.iloc[i,3])+" Hex: *")
    file_line = 0
    for line in mm_log:
        if (line_regex.search(line)):
            file_line = line
            break
         
         # Alternative flow  -  In case line is not found, look at previous minute (08:30 -> 08:29)
    if file_line == 0:
        line_regex_alternative_I = re.compile(new_format_I + r'.*?' + " Mensagem Enviada - Locomotiva: "+df_sent.iloc[i,1]+" Tipo de comunicação: 1 Número Mensagem: "+str(df_sent.iloc[i,3])+" Hex: *")
        for line in mm_log:        
            if (line_regex_alternative_I.search(line)):
                file_line = line
                logging.info('Alternative Flow I: '+str(line_regex_alternative_I))
                break

    if file_line == 0:
        # Alternative flow  -  In case line is not found, look at previous hour (08:00 -> 07:59)
        line_regex_alternative_II = re.compile(new_format_II + r'.*?' + " Mensagem Enviada - Locomotiva: "+df_sent.iloc[i,1]+" Tipo de comunicação: 1 Número Mensagem: "+str(df_sent.iloc[i,3])+" Hex: *")
        for line in mm_log:         
            if (line_regex_alternative_II.search(line)):
                file_line = line
                logging.info('Alternative Flow II: '+str(line_regex_alternative_II))
                break
       
    return(file_line) 

In [18]:
def func3(i, mm_log, file_line):
    # Get the line number in the MM log
    line_number = mm_log.index(file_line)

    return(line_number)

In [19]:
def func4(i, mm_log, line_number):
    # Remove the line from the log that was used. This will avoid duplicated lines (with same macro, same loco and timestamp)
    mm_log.pop(line_number)
    

In [20]:
def func5(file_line):
    # Get the Hex from the MM message
    file_line = file_line[file_line.find('Hex: ')+5:]
    # Removing any duplicated 'FF'
    file_line = file_line.replace("-FF","-")
    # Removing any spaces
    file_line = file_line.strip()
    return(file_line)

In [21]:
def func6(i, df_sent):
    # Transform the prefix from decimal to hexadecimal
    prefix = hex(int(df_sent.iloc[i,1]))
    prefix = prefix[2:10]
    prefix = prefix.upper()
    temp = ''              #empty string delcaration for temporary variable

    if len(prefix) == 2:
        prefix_hex = '-' + prefix + '-'
    elif len(prefix) == 1:
        prefix_hex = '-0' + prefix + '-'

    while len(prefix) > 2:     
        if len(prefix)%2 == 1:
            temp = temp + '-0' +  prefix[0] # take first character
            prefix = prefix[1:10] # take the rest
            prefix_hex =  temp + '-' + prefix + '-'
        elif len(prefix)%2 == 0:
            temp = temp + '-' + prefix[0:2] # take first two characters
            prefix = prefix[2:10] # take the rest
            prefix_hex = temp + '-' + prefix + '-'
    return(prefix_hex)

In [22]:
 def func7(prefix_hex, file_line):
    # Regex used to match relevant loglines 
    inmarsat_regex = re.compile("INMARSAT CHANNEL - Message submitted to gateway" + r'.*?' + prefix_hex + r'.*?' + file_line)
    gprs_regex = re.compile("TCP CHANNEL CONNECTION - Sending packet" + r'.*?' + prefix_hex + r'.*?' + file_line)
    
    gprs_attempt = 0
    inmarsat_attempt = 0

    # Loop over each log line
    for line in abr_log:
    # If log line matches our regex
        if (gprs_regex.search(line)):
            gprs_attempt += 1
            logging.info('ABR GPRS: '+str(line))
        elif (inmarsat_regex.search(line)):    
            inmarsat_attempt += 1
            logging.info('ABR INMARSAT: '+str(line))

    return(gprs_attempt, inmarsat_attempt)

In [23]:
def func8(i, df_sent, gprs_attempt, inmarsat_attempt):
    # Counts the amount of attempts to determine if the delay was caused by GPRS or INMARSAT channel
    if gprs_attempt > inmarsat_attempt:
        df_sent.iloc[i,13] += 1
    elif inmarsat_attempt > gprs_attempt: 
        df_sent.iloc[i,14] += 1
    elif inmarsat_attempt == gprs_attempt and (inmarsat_attempt+gprs_attempt) != 0:
        df_sent.iloc[i,13] += 1  
        df_sent.iloc[i,14] += 1 

    if inmarsat_attempt == 0 and gprs_attempt == 0:
        df_sent.iloc[i,13] = None  
        df_sent.iloc[i,14] = None
    elif (inmarsat_attempt == 1 and gprs_attempt == 0) or (inmarsat_attempt == 0 and gprs_attempt == 1):
        df_sent.iloc[i,15] += 1

    
    return(df_sent)

In [24]:

for i in range(0,len(df_sent['HM_ID_HM']),1):    
  try:
    logging.info('MESSAGE:'+str(i))
    logging.info('ID:'+str(df_sent.iloc[i,11]))
    (new_format, new_format_I, new_format_II) = func1(i,df_sent)
    (file_line) = func2(i, df_sent, new_format, mm_log, new_format_I, new_format_II)
    (line_number) = func3(i, mm_log, file_line)
    func4(i, mm_log, line_number)
    (file_line) = func5(file_line)
    (prefix_hex) = func6(i, df_sent)
    (gprs_attempt, inmarsat_attempt) = func7(prefix_hex, file_line)
    (df_sent)= func8(i, df_sent, gprs_attempt, inmarsat_attempt)
    # logging
    logging.info('FILE LINE: '+str(file_line))
    logging.info('PREFIX HEX: '+str(prefix_hex))
    logging.info('TCP: '+str(gprs_attempt)+' INMARSAT: '+str(inmarsat_attempt))
    logging.info('----------------------------------------------------------------')

  except Exception as e:
      logging.ERROR(e)
      print('ERROR OCCURED')  # If an error occurs, the channel values are being set as NULL.
      df_sent.iloc[i,13] = None  
      df_sent.iloc[i,14] = None
      break



KeyboardInterrupt: 

In [34]:
logging.info('FILE LINE: '+str(prefix_hex))

In [18]:
file_line

'03-F8-60-3C-1A-E3-02-01-04-36-34-36-30-00-00-00-00-05-4D-42-45-31-45-00'

In [32]:
prefix_hex

'-01-3E-'

In [25]:
df_sent.to_excel("testabr2.xlsx"

In [20]:
df_sent.iloc[1467]

HM_ID_HM                                                   2215160
HM_LOCO_MSG                                                    318
HM_PRF_TRM                                                    2470
HM_NUM_MSG                                                    1016
HM_DAT_OBC                                     2021-02-12 01:25:28
HM_DAT_STC                                     2021-02-12 00:41:19
HM_TXT_MSG       information de Occupations  Au tour  \n  \nAct...
HM_ENV_RCB                                                       E
HM_MSG_VOZ                                                       F
HM_NUM_FUN                                                      41
HM_SIT_ENV                                                       F
MT_ID_MT                                                   1857315
DELIVERY_TIME                                               2649.0
GSM                                                            NaN
SAT                                                           

,HM_ID_HM,HM_LOCO_MSG,HM_PRF_TRM,HM_NUM_MSG,HM_DAT_OBC,HM_DAT_STC,HM_TXT_MSG,HM_ENV_RCB,HM_MSG_VOZ,HM_NUM_FUN,HM_SIT_ENV,MT_ID_MT,DELIVERY_TIME,GSM,SAT
2229409,2214999,4304,1461,1014,2021-02-12 00:21:45,2021-02-12 00:06:35,Rapport de Erreur \n \nType do Erreur: Génér...,E,F,18,E,1857155,910.0,0,1
2229411,2215005,3045,3614,1016,2021-02-12 00:31:26,2021-02-12 00:07:51,information de Occupations Au tour \n \nAct...,E,F,41,F,1857160,1415.0,0,1
2229412,2215006,3045,3614,1016,2021-02-12 00:53:33,2021-02-12 00:07:51,information de Occupations Au tour \n \nAct...,E,F,41,F,1857161,2742.0,0,1
2229416,2215022,4304,1461,1003,2021-02-12 00:22:42,2021-02-12 00:13:13,Solicitation de Réduction de Autorisation \n ...,E,F,5,E,1857177,569.0,0,1
2229713,2215071,3045,3614,1016,2021-02-12 01:15:39,2021-02-12 00:21:26,information de Occupations Au tour \n \nAct...,E,F,41,F,1857226,3253.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215300,2219571,3033,4632,1016,2021-02-13 00:44:51,2021-02-12 23:44:49,information de Occupations Au tour \n \nAct...,E,F,41,F,1861379,3602.0,0,0
2215564,2219611,4309,4611,1006,2021-02-12 23:57:13,2021-02-12 23:48:47,Notificationde croisement du Train \n \nPréf...,E,F,17,E,1861419,506.0,0,0
2215571,2219620,3033,4632,1006,2021-02-13 00:49:52,2021-02-12 23:49:34,Notificationde croisement du Train \n \nPréf...,E,F,17,F,1861428,3618.0,0,0
2215580,2219639,3033,4632,1016,2021-02-13 00:52:53,2021-02-12 23:52:24,information de Occupations Au tour \n \nAct...,E,F,41,F,1861448,3629.0,0,0
